# AI Assistant with LLama2 7B
### In Sagemaker Studio use Image: Data Science 3.0, Instance Type ml.m5.large

In this notebook you'll make an AI assistant to help customers make the best decision about getting a new shoe from Cloudrunners Shoes. You will also take a short dive into practical token limitations for the model you are working with, and where you can focus your learning in the case you need to build more powerful AI agent systems.

## Setup

***

In [6]:
%pip install --upgrade --quiet sagemaker

Note: you may need to restart the kernel to use updated packages.


***
You can continue with the default model or choose a different model: this notebook will run with the following model IDs :
- `meta-textgeneration-llama-2-7b-f`
- `meta-textgeneration-llama-2-13b-f`
- `meta-textgeneration-llama-2-70b-f`
***

In [7]:
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "2.*"

instance_type = "ml.g5.xlarge"

## Deploy model (this will take 10-15 minutes)

***
You can now deploy the model using SageMaker JumpStart.
***

In [8]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, instance_type=instance_type)

predictor = model.deploy(accept_eula=True)

------------!

In [9]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

In [10]:
endpoint_name = predictor.endpoint_name

import boto3, json
sm_client = boto3.client('sagemaker-runtime')

def format_messages(messages) :
    """Format messages for Llama-2 chat models.
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)

def transform_input(prompt: str, temperature) -> bytes:
    test = [{"role": "user", "content": prompt}]
    input_str = json.dumps({
        "inputs": format_messages(test),
        "parameters": {"max_new_tokens": 512,"top_p": 0.9,"temperature": temperature}
    })
    return input_str.encode('utf-8')


def transform_output(output: bytes) -> str:
    response_json = json.loads(output['Body'].read().decode("utf-8"))
    return response_json[0]["generated_text"]

def generate(prompt, temperature = 0.6):
    response = sm_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        CustomAttributes="accept_eula=true", 
        ContentType="application/json",
        Body=transform_input(prompt, temperature)
    )
    response_output = transform_output(response)
    return response_output

### Edit System Message

### Include One-to-many Shot Learning

In [11]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [12]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## Data

### Cloudrunners Shoes Details

In [13]:
shoes = [
    {
        "model": "Sky Glider",
        "type": "Running", 
        "features": {
            "upper": "Mesh",
            "sole": "EVA foam",
            "lacing": "Quick-tie",
            "drop": "8mm",
            "color": "Blue with yellow accents"
        },
        "usps": ["Lightweight", "Responsive cushioning", "Seamless upper"],
        "price": 119.95,
        "internal_id": "SG123",
        "weight": "220g",
        "manufacturer_location": "Vietnam"
    },
    {   
        "model": "Trail Trekker",
        "type": "Hiking",
        "features": {
            "upper": "Synthetic leather",
            "sole": "Rubber lug", 
            "lacing": "Traditional",
            "drop": "12mm",
            "color": "Khaki green"
        },
        "usps": ["Rugged construction", "Super grippy sole", "Waterproof"], 
        "price": 129.99,
        "internal_id": "TT321",
        "weight": "340g",
        "manufacturer_location": "China"
    },
    {
        "model": "Marathon Master",
        "type": "Racing",
        "features": {
            "upper": "Mesh",
            "sole": "Carbon fiber plate",
            "lacing": "Speed laces",
            "drop": "6mm",
            "color": "Neon yellow and black"
        },
        "usps": ["Maximizes energy return", "Lightning fast", "Seamless comfort"],
        "price": 179.50, 
        "internal_id": "MM111",
        "weight": "180g",
        "manufacturer_location": "USA"
    }
]

## Shoes AI Assistant

In [14]:
system_message = """
You are a friendly chatbot knowledgeable about shoes. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [15]:
print(chatbot.chat("Can you tell me about the latest models?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot knowledgeable about shoes. When asked about specific shoe models or features, you try to provide accurate and helpful answers. Your goal is to assist and inform potential customers to the best of your ability.
<</SYS>>
Can you tell me about the latest models? [/INST] [/INST]  Of course! I'd be happy to help you with the latest shoe models. There are so many great options out there, and I'm sure I can find something that suits your style and preferences.

For men, some of the latest trends include:

1. Sneakers: The classic basketball shoes have evolved into stylish and comfortable lifestyle shoes. Look for bold colors, eye-catching designs, and premium materials like leather and suede.
2. Boat shoes: These lace-up shoes are perfect for casual, nautical-inspired looks. Look for boat shoes with leather or suede uppers, and consider adding a touch of color with contrasting laces or a bold sole.
3. Loafers: These slip-on shoes are grea

In [16]:
chatbot.reset()

## Cloudrunners Shoes AI Assistant

In [17]:
system_message = f"""
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes {shoes}. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [18]:
print(chatbot.chat("Can you tell me about the latest models?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes [{'model': 'Sky Glider', 'type': 'Running', 'features': {'upper': 'Mesh', 'sole': 'EVA foam', 'lacing': 'Quick-tie', 'drop': '8mm', 'color': 'Blue with yellow accents'}, 'usps': ['Lightweight', 'Responsive cushioning', 'Seamless upper'], 'price': 119.95, 'internal_id': 'SG123', 'weight': '220g', 'manufacturer_location': 'Vietnam'}, {'model': 'Trail Trekker', 'type': 'Hiking', 'features': {'upper': 'Synthetic leather', 'sole': 'Rubber lug', 'lacing': 'Traditional', 'drop': '12mm', 'color': 'Khaki green'}, 'usps': ['Rugged construction', 'Super grippy sole', 'Waterproof'], 'price': 129.99, 'internal_id': 'TT321', 'weight': '340g', 'manufacturer_location': 'China'}, {'model': 'Marathon Master', 'type': 'Racing', 'features': {'upper': 'Mesh', 'sole': 'Carbon fiber plate', 'lacing': 'Speed laces', 'drop': '6mm', 'color': 'Neon yellow and black'}, 'usps': ['Maximizes energy retur

In [19]:
print(chatbot.chat("How much do each of the models cost?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes [{'model': 'Sky Glider', 'type': 'Running', 'features': {'upper': 'Mesh', 'sole': 'EVA foam', 'lacing': 'Quick-tie', 'drop': '8mm', 'color': 'Blue with yellow accents'}, 'usps': ['Lightweight', 'Responsive cushioning', 'Seamless upper'], 'price': 119.95, 'internal_id': 'SG123', 'weight': '220g', 'manufacturer_location': 'Vietnam'}, {'model': 'Trail Trekker', 'type': 'Hiking', 'features': {'upper': 'Synthetic leather', 'sole': 'Rubber lug', 'lacing': 'Traditional', 'drop': '12mm', 'color': 'Khaki green'}, 'usps': ['Rugged construction', 'Super grippy sole', 'Waterproof'], 'price': 129.99, 'internal_id': 'TT321', 'weight': '340g', 'manufacturer_location': 'China'}, {'model': 'Marathon Master', 'type': 'Racing', 'features': {'upper': 'Mesh', 'sole': 'Carbon fiber plate', 'lacing': 'Speed laces', 'drop': '6mm', 'color': 'Neon yellow and black'}, 'usps': ['Maximizes energy retur

In [20]:
print(chatbot.chat("I'm torn between the Sky Glider and the Marathon Master, how should I decide which is best or me?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes [{'model': 'Sky Glider', 'type': 'Running', 'features': {'upper': 'Mesh', 'sole': 'EVA foam', 'lacing': 'Quick-tie', 'drop': '8mm', 'color': 'Blue with yellow accents'}, 'usps': ['Lightweight', 'Responsive cushioning', 'Seamless upper'], 'price': 119.95, 'internal_id': 'SG123', 'weight': '220g', 'manufacturer_location': 'Vietnam'}, {'model': 'Trail Trekker', 'type': 'Hiking', 'features': {'upper': 'Synthetic leather', 'sole': 'Rubber lug', 'lacing': 'Traditional', 'drop': '12mm', 'color': 'Khaki green'}, 'usps': ['Rugged construction', 'Super grippy sole', 'Waterproof'], 'price': 129.99, 'internal_id': 'TT321', 'weight': '340g', 'manufacturer_location': 'China'}, {'model': 'Marathon Master', 'type': 'Racing', 'features': {'upper': 'Mesh', 'sole': 'Carbon fiber plate', 'lacing': 'Speed laces', 'drop': '6mm', 'color': 'Neon yellow and black'}, 'usps': ['Maximizes energy retur

In [22]:
chatbot.reset()

In [23]:
print(chatbot.chat("I'm torn between the Marathon Master and the Trail Trekker, how should I decide which is best or me?"))

<s>[INST] <s>[INST] <<SYS>>
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes [{'model': 'Sky Glider', 'type': 'Running', 'features': {'upper': 'Mesh', 'sole': 'EVA foam', 'lacing': 'Quick-tie', 'drop': '8mm', 'color': 'Blue with yellow accents'}, 'usps': ['Lightweight', 'Responsive cushioning', 'Seamless upper'], 'price': 119.95, 'internal_id': 'SG123', 'weight': '220g', 'manufacturer_location': 'Vietnam'}, {'model': 'Trail Trekker', 'type': 'Hiking', 'features': {'upper': 'Synthetic leather', 'sole': 'Rubber lug', 'lacing': 'Traditional', 'drop': '12mm', 'color': 'Khaki green'}, 'usps': ['Rugged construction', 'Super grippy sole', 'Waterproof'], 'price': 129.99, 'internal_id': 'TT321', 'weight': '340g', 'manufacturer_location': 'China'}, {'model': 'Marathon Master', 'type': 'Racing', 'features': {'upper': 'Mesh', 'sole': 'Carbon fiber plate', 'lacing': 'Speed laces', 'drop': '6mm', 'color': 'Neon yellow and black'}, 'usps': ['Maximizes energy retur

### Clean up the endpoint

In [24]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Make SURE the endpoint was deleted, you can check in the sagemaker UI as well. 
### Check here https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints
## If the endpoint is not deleted, it will consume all of your budget